In [14]:
import gplace

location = "13.744677,100.5295593"  # Latitude and Longitude
keyword = "ร้านกาแฟ"
result = gplace.nearby_search(keyword, location)

In [15]:
def find_place_from_text(location:str):
    """Finds a place and related data from the query text"""
    
    result = gplace.find_place_from_text(location)
    r = result['candidates'][0]
    return f"""
    address: {r['formatted_address']}\n
    location: {r['geometry']['location']}\n
    name: {r['name']}\n
    opening hours: {r['opening_hours']}\n
    rating: {r['rating']}\n
    """
    
def nearby_search(keyword:str, location:str, radius=2000, place_type=None):
    """Searches for many places nearby the location based on a keyword. using keyword like \"coffee shop\", \"restaurants\". radius is the range to search from the location"""
    location = gplace.find_location(location, radius=radius)
    result = gplace.nearby_search(keyword, location, radius)
    
    strout = ""
    for r in result:
        # Use .get() to handle missing keys
        address = r.get('vicinity', 'N/A')
        location_info = r.get('geometry', {}).get('location', 'N/A')
        name = r.get('name', 'N/A')
        opening_hours = r.get('opening_hours', 'N/A')
        rating = r.get('rating', 'N/A')
        plus_code = r.get('plus_code', {}).get('global_code', 'N/A')
        
        strout += f"""
        address: {address}\n
        location: {location_info}\n
        name: {name}\n
        opening hours: {opening_hours}\n
        rating: {rating}\n
        plus code: {plus_code}\n\n
        """
    return strout

def nearby_dense_community(location:str, radius:int=2000):
    """ getting nearby dense community such as (community mall, hotel, school, etc), by geomatric location, radius(in meters)
    return list of location community nearby, name, community type.
    """
    result = gplace.nearby_dense_community(location, radius)
    
    strout = ""
    for r in result:
        # Use .get() to handle missing keys
        address = r.get('vicinity', 'N/A')
        location_info = r.get('geometry', {}).get('location', 'N/A')
        name = r.get('name', 'N/A')
        opening_hours = r.get('opening_hours', 'N/A')
        rating = r.get('rating', 'N/A')
        plus_code = r.get('plus_code', {}).get('global_code', 'N/A')
        
        strout += f"""
        name: {address}\n
        types: {location_info}\n
        """
    return strout

In [16]:
# gplace_tools.py
from langgraph.prebuilt import ToolNode
from langchain_core.tools import tool
from langchain_core.tools import Tool
from langchain_google_community import GoogleSearchAPIWrapper
import utils

utils.load_env()

search = GoogleSearchAPIWrapper()

find_place_from_text = tool(find_place_from_text)
nearby_search = tool(nearby_search)
google_search = Tool(
    name="google_search",
    description="Search Google for recent results.",
    func=search.run,
)

tools = [find_place_from_text, nearby_search]

# Create ToolNodes for each tool
tool_node = ToolNode(tools)